# Think Green Investing 

In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import panel as pn



pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from finta import TA
import talib as ta 

%matplotlib inline

## Data Preparation

In [2]:
# Initializing alpaca trade api
load_dotenv()
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
alpaca=tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version='v2')

In [3]:
# function that pulls tickers daily closing prices in the given time period using alpaca trade api, 
# Initialization of the api is required before using.
# It returns a dataframe with the tickers, and closing prices as two level column structured dataframe with index defined as date  

def get_tickers_data(tickers, start_date, end_date, timeframe):
    start_date = pd.Timestamp(start_date, tz = "America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz = "America/New_York").isoformat()
    tickers_data_df = alpaca.get_bars(tickers, timeframe,start_date , end_date, adjustment = 'raw').df
    tickers_data_df.index = tickers_data_df.index.date
    tickers_data_df = tickers_data_df[['symbol', 'open', 'high', 'low','close']]
    tickers_data_list=[]
    for ticker in tickers:
        tickers_data_list.append(tickers_data_df[tickers_data_df['symbol']==ticker].drop('symbol', axis=1))
        
    clean_tickers_data = pd.concat(tickers_data_list,axis=1, keys=tickers)
    #clean_tickers_data.columns=clean_tickers_data.columns.droplevel(-1)
    return clean_tickers_data

In [4]:
# Defining the stocks in the solar energy portfolio form top performers in 2021 using Motley fool and Investopedia. The companies are:
#1.First Solar (FSLR)
#2.Solar Edge Technologies (SEDG)
#3.Daqo New Energy Corp. (DQ)
#4.Canadian Solar INC (CSIQ)
#5.Algonquin Power & Utilities Corp (AQN)

# Initialize data

# Set the ticker information
portfolio = ['FSLR' , 'SEDG', 'DQ', 'CSIQ', 'AQN']
indices = ['SPY']
timeframe='1Day'
tickers = portfolio + indices

### Set the Period: from Jan 2019 to June 2022

In [5]:
# Setting up period and pulling data and cleaning it. 
start_date = '2019-01-01'
end_date = '2022-06-15'

tickers_data = get_tickers_data(tickers, start_date, end_date, timeframe) 
tickers_data.dropna(inplace = True)

In [6]:
#function to decide the buy and sell signals based on the indicator RSI and Bollinger Bands.
def algo_trading(close_df): 
   
    # Determine the Bollinger Bands for the Dataset
    bbands_df = TA.BBANDS(close_df)

    # Concatenate the Bollinger Bands to the DataFrame
    bb_df = pd.concat([close_df, bbands_df], axis=1)

    # Set the Signal column
    bb_df["Signal"] = 0.0
# RSI calculation
    bb_df['RSI'] = ta.RSI(bb_df['close'], 14)
# Generate the trading signals 1 (entry) or -1 (exit) for a long position trading algorithm 
    for index, row in bb_df.iterrows():
        if row["close"] < row["BB_LOWER"] and row["RSI"] < row["BB_LOWER"]:
            bb_df.loc[index, "Signal"] = 1.0
        if row["close"] > row["BB_UPPER"] and row["RSI"] > row["BB_UPPER"]:
            bb_df.loc[index,"Signal"] = -1.0
    return bb_df


In [7]:
# Visualize entry position relative to close price
def visualize_algo_trading(bb_df): 

    entry = bb_df[bb_df["Signal"] == 1.0]["close"].hvplot.scatter(
        color="green",
        marker="^",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize exit position relative to close price
    exit = bb_df[FSLR_bb_df["Signal"] == -1.0]["close"].hvplot.scatter(
        color="red",
        marker="v",
        size=200,
        legend=False,
        ylabel="Price in $",
        width=1000,
        height=400
    )

# Visualize close price for the investment
    security_close = bb_df[["close"]].hvplot(
        line_color="gray",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_upper = bb_df[["BB_UPPER"]].hvplot(
        line_color="purple",
        ylabel="Price in $",
        width=1000,
        height=400
    )


    bb_middle = bb_df[["BB_MIDDLE"]].hvplot(
        line_color="orange",
        ylabel="Price in $",
        width=1000,
        height=400
    )

    bb_lower = bb_df[["BB_LOWER"]].hvplot(
        line_color="blue",
        ylabel="Price in $",
        width=1000,
        height=400
    )


# Overlay plots
    bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
    return  bbands_plot

In [8]:
#function to calculate returns based on initial capital and signals establised from the trading algorithm. 
def cal_return(initial_capital,bb_df):
   
    # Set the share size
    share_size = 100

    # Buy a 100 share position when the dual moving average crossover Signal equals 1
    # Otherwise, `Position` should be zero (sell)
    bb_df['Position'] = share_size * bb_df['Signal']

    # Determine the points in time where a 100 share position is bought or sold
    bb_df['Entry/Exit Position'] = bb_df['Position'].diff()

    # Multiply the close price by the number of shares held, or the Position
    bb_df['Portfolio Holdings'] = bb_df['close'] * bb_df['Position']

    # Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
    bb_df['Portfolio Cash'] = initial_capital - (bb_df['close'] * bb_df['Entry/Exit Position']).cumsum() 

    # Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
    bb_df['Portfolio Total'] = bb_df['Portfolio Cash'] + bb_df['Portfolio Holdings']

    # Calculate the portfolio daily returns
    bb_df['Portfolio Daily Returns'] = bb_df['Portfolio Total'].pct_change()

    # Calculate the portfolio cumulative returns
    bb_df['Portfolio Cumulative Returns'] = (1 + bb_df['Portfolio Daily Returns']).cumprod() - 1


    return bb_df

## First Solar Algorithm 

In [9]:
capital = 1e5
FSLR_bb_df = algo_trading(tickers_data['FSLR'])
plot_tr=visualize_algo_trading(FSLR_bb_df)   
FSLR_bb_df = cal_return(capital,FSLR_bb_df)


In [10]:
print(FSLR_bb_df['Portfolio Total'][-1])
plot_tr

104078.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Solar Edge Technologies

In [11]:
capital = 1e5
SEDG_bb_df = algo_trading(tickers_data['SEDG'])
plot_tr=visualize_algo_trading(SEDG_bb_df)   
SEDG_bb_df = cal_return(capital,SEDG_bb_df)


In [12]:
print(SEDG_bb_df['Portfolio Total'][-1])
plot_tr

99902.5


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Daqo New Energy Corp.

In [13]:
capital = 1e5
DQ_bb_df = algo_trading(tickers_data['DQ'])
plot_tr=visualize_algo_trading(DQ_bb_df)   
DQ_bb_df = cal_return(capital,DQ_bb_df)


In [14]:
print(DQ_bb_df['Portfolio Total'][-1])
plot_tr

100616.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Canadian Solar INC 

In [15]:
capital = 1e5
CSIQ_bb_df = algo_trading(tickers_data['CSIQ'])
plot_tr=visualize_algo_trading(CSIQ_bb_df)   
CSIQ_bb_df = cal_return(capital,CSIQ_bb_df)


In [16]:
print(CSIQ_bb_df['Portfolio Total'][-1])
plot_tr

100361.73


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Algonquin Power & Utilities Corp. 

In [17]:
capital = 1e5
AQN_bb_df = algo_trading(tickers_data['AQN'])
plot_tr=visualize_algo_trading(AQN_bb_df)   
AQN_bb_df = cal_return(capital,AQN_bb_df)


In [18]:
print(AQN_bb_df['Portfolio Total'][-1])
plot_tr

99823.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## SPY

In [19]:
capital = 1e5
SPY_bb_df = algo_trading(tickers_data['SPY'])
plot_tr=visualize_algo_trading(SPY_bb_df)   
SPY_bb_df = cal_return(capital,SPY_bb_df)


In [20]:
print(SPY_bb_df['Portfolio Total'][-1])
plot_tr

106960.0


:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

## Green Stocks Portfolio

In [21]:
tickers_data.drop(columns='SPY',inplace=True)


In [22]:
tickers_data.columns=tickers_data.columns.droplevel(0)

In [23]:
#portfolio based on equal weight to each stock


tickers_data['Portfolio_open'] = capital + tickers_data['open'].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_high'] = capital + tickers_data['high'].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_low'] = capital + tickers_data["low"].pct_change().cumsum().mean(axis = 1) *capital
tickers_data['Portfolio_close'] = capital + tickers_data["close"].pct_change().cumsum().mean(axis = 1) *capital
portfolio_df=pd.concat([tickers_data['Portfolio_open'],tickers_data['Portfolio_high'],tickers_data['Portfolio_low'], tickers_data['Portfolio_close']], keys=['open','high','low','close'], axis=1)
portfolio_df.dropna(inplace=True)
Portfolio_bb_df = algo_trading(portfolio_df)
plot_tr=visualize_algo_trading(Portfolio_bb_df)   
Portfolio_bb_df = cal_return(capital,Portfolio_bb_df)


<ipython-input-7-8485c580353c>:15: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [24]:
Portfolio_bb_df['Portfolio Total']
plot_tr

:Overlay
   .Curve.I          :Curve   [index]   (close)
   .Curve.II         :Curve   [index]   (BB_UPPER)
   .Curve.III        :Curve   [index]   (BB_MIDDLE)
   .Curve.IV         :Curve   [index]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [index]   (close)
   .Scatter.Close.II :Scatter   [index]   (close)

In [25]:
print(Portfolio_bb_df['Portfolio Total'][-1])

2816781.599228788
